In [64]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os
from dotenv import load_dotenv
load_dotenv()

True

In [65]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [66]:
#extract pdf data

def load_pdf(data):
    loader = DirectoryLoader(data,glob="*pdf",
                    loader_cls=PyPDFLoader)
    docs = loader.load()
    return docs

In [67]:
extracted_data = load_pdf(r"data")

In [68]:
#create text chunks

def text_splitter(extracted_data):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
    chunks = splitter.split_documents(extracted_data)
    return chunks

In [69]:
text_chunks = text_splitter(extracted_data)
len(text_chunks)

7486

In [70]:
#download embedding model
def download_hugging_face_embedding():
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding

# embedding = download_hugging_face_embedding()

In [71]:
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x00000217E483A550>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x00000217A6C79850>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [72]:
query_result = embedding.embed_query("Hello")
len(query_result)

1536

In [73]:
# PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY',"aecd0884-1eb4-4353-80b0-a9b0fa930ca0")

In [74]:
# from pinecone import Pinecone as PC
# from pinecone import ServerlessSpec
# pc = PC(api_key= PINECONE_API_KEY, environment = 'us-east-1')

In [75]:
# index_name="medicalbot"
# index = pc.Index(index_name)
# index

In [76]:
# from langchain_pinecone import Pinecone
# docsearch = Pinecone.from_texts([t.page_content for t in text_chunks] , embedding, index_name = index_name)

In [77]:
from langchain_openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings()
len(embedding.embed_query("Hello"))

1536

In [ ]:
from langchain_chroma import Chroma
persist_directory="db"
vectorstore = Chroma.from_documents(
    text_chunks,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
vectorstore.similarity_search("what are allergies",k=3)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vectorstore.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input = input(f"Input Prompt : ")
    result = qa({"query" : user_input})
    print("Response : ", result["result"])

c:\Users\hp\.conda\envs\GENAI\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response :  Skin disease refers to any medical condition that affects the skin, including but not limited to acne, eczema, psoriasis, and skin cancer.
Response :  Cancer is defined as the uncontrolled growth of cells, with loss of differentiation and commonly, with metastasis, spread of the cancer to other tissues and organs. Cancers are malignant growths that can arise in any tissue or organ of the body.
Response :  There are no bruises in the woman's eye.
